# Extra Notebook: Resolution of Images

In [1]:
# Imports
from dsa_helpers.girder_utils import login
from tqdm.notebook import tqdm
from multiprocessing import Pool
from girder_client import HttpError
import pandas as pd

In [2]:
# Authenticate girder client.
gc = login("https://computablebrain.emory.edu/api/v1")

In [ ]:
# Get all the items from Emory.
items = gc.get(
    f"resource/6382af947f8a5e686a682eab/items?type=folder&limit=0&offset=0&sort=_id&sortdir=1"
)

items.extend(
    gc.get(
        f"resource/638148727f8a5e686a5db6e6/items?type=folder&limit=0&offset=0&sort=_id&sortdir=1"
    )
)

print(f"Total number of items: {len(items)}")

In [ ]:
def get_resolution(item):
    try:
        large_image_metadata = gc.get(f"item/{item['_id']}/tiles")
        
        return item['name'], large_image_metadata["mm_x"], large_image_metadata["mm_y"]
    except HttpError:
        return None
    
    
resolutions = []

with Pool(20) as pool:
    jobs = [
        pool.apply_async(get_resolution, args=(item,))
        for item in items
    ]
    
    for job in tqdm(jobs):
        result = job.get()
        
        if result is not None:
            resolutions.append(result)

In [ ]:
# Convert to a dataframe.
df = pd.DataFrame(resolutions, columns=["name", "mm_x", "mm_y"])

# Scale the coordinates to micrometers.
df["um_x"] = df["mm_x"] * 1000
df["um_y"] = df["mm_y"] * 1000

df.to_csv("csvs/emory_resolutions.csv", index=False)
df.head()